# Guided Project: Answering Business Questions Using SQL

In this project we are going to use SQL to answer some business questions about [Chinook](https://github.com/lerocha/chinook-database).

The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.
The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases.

## Knowing our Data
This information is contained in eleven tables, a schema diagram will help us understand the available columns and the structure of the data.
![schema](./chinook_schema.png)
### Connecting to database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Database Tables
Getting a list of all tables and views in our database:

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


As per our schema we have 11 tables in our data base.
let's take a look at one table and the data it contains about artists in our store. 

In [4]:
%%sql
SELECT *
FROM artist
LIMIT 5;

Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


we have in artist table 2 columns:
- <mark>artist_id</mark> unique id for each artist.
- <mark>name</mark> name of the artist.

# First Question, adding new albums:

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four.

All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre |
| ----------- | ----------- |
| Regal | Hip-Hop |
| Red Tone | Punk | 
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues | 

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of our findings, and make a recommendation for the three artists whose albums we should purchase for the store.

## Genres and their sold tracks in USA

In [5]:
%%sql
WITH genre_tracks_usa AS
    (SELECT 
         g.genre_id genre_id,
         g.name genre_name,
         t.track_id track_id,
         c.country country,
         il.quantity amount_track_sold
     FROM genre g
     LEFT JOIN track t
         ON g.genre_id=t.genre_id
     LEFT JOIN invoice_line il
         ON t.track_id=il.track_id
     LEFT JOIN invoice i
         ON il.invoice_id = i.invoice_id
     LEFT JOIN customer c
         ON i.customer_id=c.customer_id
    )

SELECT 
    genre_id,
    genre_name,
    SUM(amount_track_sold) number_tracks_sold,
    SUM(amount_track_sold)/CAST(
                            (SELECT SUM(amount_track_sold)
                             FROM genre_tracks_usa
                             WHERE country="USA"
                             GROUP BY country) AS FLOAT) sold_percentage
FROM genre_tracks_usa gtu
WHERE country="USA"
GROUP BY genre_id
ORDER BY 3 DESC
LIMIT 10;


Done.


genre_id,genre_name,number_tracks_sold,sold_percentage
1,Rock,561,0.5337773549000951
4,Alternative & Punk,130,0.12369172216936251
3,Metal,124,0.11798287345385347
14,R&B/Soul,53,0.05042816365366318
6,Blues,36,0.03425309229305423
23,Alternative,35,0.03330161750713606
7,Latin,22,0.02093244529019981
9,Pop,22,0.02093244529019981
17,Hip Hop/Rap,20,0.019029495718363463
2,Jazz,14,0.013320647002854425


The three artists whose albums we should purchase for the store, based on sales of tracks from their genres across USA before are:
* Red Tone (Punk) 
* Meteor and the Girls (Pop)
* Slim Jim Bites (Blues) 

However we should keep our eyes open for releases of Rock from the new record label. As sales of rock generate more than 53% of our sold genres in USA alone.

# Second Question, analyzing employee sales performance:

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
%%sql
WITH number_customers AS
    (SELECT 
     support_rep_id,
     COUNT(customer_id) number_customers    
     FROM customer
     GROUP BY support_rep_id)
    
    
SELECT 
    e.employee_id employee_id,
    e.first_name||' '||e.last_name Sales_Support_Agent,
    e.hire_date hire_date,
    e.city city,
    e.state state,
    e.country country,
    n_c.number_customers number_customers,
    SUM(i.total) total_sales
FROM employee e
LEFT JOIN customer c 
    ON e.employee_id=c.support_rep_id
LEFT JOIN invoice i
    ON c.customer_id=i.customer_id
LEFT JOIN number_customers n_c
    ON e.employee_id=n_c.support_rep_id
WHERE title="Sales Support Agent"
GROUP BY e.employee_id
ORDER BY total_sales DESC;

Done.


employee_id,Sales_Support_Agent,hire_date,city,state,country,number_customers,total_sales
3,Jane Peacock,2017-04-01 00:00:00,Calgary,AB,Canada,21,1731.510000000004
4,Margaret Park,2017-05-03 00:00:00,Calgary,AB,Canada,20,1584.0000000000032
5,Steve Johnson,2017-10-17 00:00:00,Calgary,AB,Canada,18,1393.9200000000028


Our top sales support agent is Jane Peacock with a difference from our bottom sales support agent of almost 25%, let's explore why the reason for that is.

Location of work is not the reason since they are at the same city, however there are difference in number of customer they are supporting 21 against 18, another reason is the hiring date at the store with a difference of half a year between them.

let's explore deeper developing a KPI of their performance against number of customers they support.

In [7]:
%%sql
WITH number_customers AS
    (SELECT 
     support_rep_id,
     COUNT(customer_id) number_customers    
     FROM customer
     GROUP BY support_rep_id),   
    employee_sales AS
    (SELECT 
        e.employee_id employee_id,
        e.first_name||' '||e.last_name Sales_Support_Agent,
        e.hire_date hire_date,
        e.city city,
        e.state state,
        e.country country,
        n_c.number_customers number_customers,
        SUM(i.total) total_sales
    FROM employee e
    LEFT JOIN customer c 
        ON e.employee_id=c.support_rep_id
    LEFT JOIN invoice i
        ON c.customer_id=i.customer_id
    LEFT JOIN number_customers n_c
        ON e.employee_id=n_c.support_rep_id
    WHERE title="Sales Support Agent"
    GROUP BY e.employee_id
    ORDER BY total_sales DESC)

SELECT 
    employee_id,
    Sales_Support_Agent,
    total_sales/number_customers sales_per_customer  
FROM employee_sales
ORDER BY 3 DESC;

Done.


employee_id,Sales_Support_Agent,sales_per_customer
3,Jane Peacock,82.45285714285734
4,Margaret Park,79.20000000000016
5,Steve Johnson,77.44000000000015


Still here Jane Peacock	is our top performer, we can evaluate their performance deeper next quarter against this quarter performance and check if their performance is improving taking into consideration their time in the store.
# Third Question, analyzing sales by country:
Our next task is to analyze the sales data for customers from each different country.

In particular, we are going to calculate data, for each country, on the:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are a number of countries with only one customer, we will group these customers as "Other" in our analysis. 

In [8]:
%%sql
WITH country_customers AS
    (SELECT
         CASE 
             WHEN(SELECT COUNT(*)
                  FROM customer
                  WHERE country=c.country)>1 THEN c.country
             ELSE "Other"
         END AS country,
         CASE 
             WHEN(SELECT COUNT(*)
                  FROM customer
                  WHERE country=c.country)>1 THEN 0
             ELSE 1
         END AS other,
         c.customer_id,
         i.invoice_id,
         i.total
    FROM customer c
    INNER JOIN invoice i
        ON c.customer_id=i.customer_id)
       
SELECT 
    country,
    COUNT(DISTINCT customer_id) number_customers,
    SUM(total) sales,
    SUM(total)/COUNT(DISTINCT customer_id) customer_lifetime_value,
    SUM(total)/COUNT(DISTINCT invoice_id) average_order
FROM country_customers
GROUP BY country
ORDER BY other, average_order DESC;

Done.


country,number_customers,sales,customer_lifetime_value,average_order
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Germany,4,334.62,83.655,8.161463414634147
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


Based on our summary about sales across countries, the average customer order in Czech Republic, United Kingdom and India spend the most per order. so getting more orders from customers in those countries at our store can boost our sales more.

It must be noted that the amount of data from each of those countries is relatively low giving us low confidence in concluding findings. we have to proceed with our marketing campaign budget cautiously.

A better recommendation would be start small marketing campaigns in those countries and then analysing the results further to make better understanding of customers in those locations.

# Last Question, Albums vs Individual Tracks:

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

We have a couple of 'edge cases' which prevent us from getting a 100% accurate answer to our question. In this instance, we have two edge cases to consider:
* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

## Our approach:
We are going to compare album purchases invoices volume to track purchases invoices volume. and with that we could gauge the purchases through our store to make an informed decision.

In [24]:
%%sql
WITH invoice_tracks AS
    (SELECT
         il.invoice_id,
         il.track_id,
         t.album_id
     FROM invoice_line il
     INNER JOIN track t
         ON il.track_id=t.track_id
    ),
    invoice_info AS
    (
    SELECT 
        i_t.*,
        CASE 
            WHEN (
                ((SELECT i_t1.track_id 
                 FROM invoice_tracks i_t1
                 WHERE i_t1.invoice_id=i_t.invoice_id
                 EXCEPT
                 SELECT t1.track_id 
                 FROM track t1
                 WHERE t1.album_id=i_t.album_id) IS NULL
                 AND
                 (             
                 SELECT t2.track_id 
                 FROM track t2
                 WHERE t2.album_id=i_t.album_id
                 EXCEPT
                 SELECT i_t2.track_id 
                 FROM invoice_tracks i_t2
                 WHERE i_t2.invoice_id=i_t.invoice_id) IS NULL)
                 ) THEN "album"
            ELSE "tracks"
        END AS purchase_type
    FROM invoice_tracks i_t),
    invoice_type AS
    (SELECT 
         DISTINCT invoice_id,
         purchase_type         
    FROM invoice_info)
    
SELECT 
    purchase_type,
    COUNT(invoice_id) invoices_volume,
    (CAST(COUNT(invoice_id) AS FLOAT)/(SELECT COUNT(invoice_id)
                                       FROM invoice))*100 invoices_percentage
FROM invoice_type
GROUP BY purchase_type

Done.


purchase_type,invoices_volume,invoices_percentage
Album,114,18.566775244299674
tracks,500,81.43322475570032


Looking at the results, the vast majority of sales are tracks about four fifths of invoices are tracks and almost a fifth are album sales. so a considerate amount of our sales are still albums. 

A recommendation against only buying tracks from record companies is the better decision for the time being so as not to lose the sales of albums to competitors.